# Classification with Gated Residual and Variable Selection Networks

**Author:** [Mike Fournigault](https://www.linkedin.com/in/mike-fournigault-57312071/)<br>
**Description:** Using Gated Residual and Variable Selection Networks for quality assessment of astronomical images.
Freely inspired by the work of [Khalid Salama](https://keras.io/examples/structured_data/classification_with_grn_and_vsn/)

## Introduction

This example demonstrates the use of Gated
Residual Networks (GRN) and Variable Selection Networks (VSN), proposed by
Bryan Lim et al. in
[Temporal Fusion Transformers (TFT) for Interpretable Multi-horizon Time Series Forecasting](https://arxiv.org/abs/1912.09363),
for structured data classification. GRNs give the flexibility to the model to apply
non-linear processing only where needed. VSNs allow the model to softly remove any
unnecessary noisy inputs which could negatively impact performance.
Together, those techniques help improving the learning capacity of deep neural
network models.

Note that this example implements only the GRN and VSN components described in
in the paper, rather than the whole TFT model, as GRN and VSN can be useful on
their own for structured data learning tasks.


To run the code you need to use TensorFlow 2.3 or higher.

In [1]:
import os
import sys

# Only the TensorFlow backend supports string inputs.
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers

sys.path.append(os.path.abspath("../../src/"))
import mapping_fits_objects as mfo

2025-03-21 13:45:36.586927: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-21 13:45:36.587061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-21 13:45:36.607024: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 13:45:36.661848: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 13:45:38.146717: W tensorflow/compiler/tf2

## Loading and preparing the datasets

Reading and merging catalog and mapping files

In [2]:
columns = ["OBJECT_ID", "FITS_ID", "CCD_ID", "ISO0", "BACKGROUND", "ELLIPTICITY", "ELONGATION", "CLASS_STAR", "FLAGS", "EXPTIME"]
data_path = "../../data/"
proc_path = os.path.join(data_path, "processed")
fm_path = os.path.join(data_path, "for_modeling")


In [3]:
def custom_reader_func(datasets):
  return datasets.interleave(lambda x: x, num_parallel_calls=tf.data.AUTOTUNE)


In [4]:
# Load the tensorflow Datasets
print("current directory: ", os.getcwd())
print("Content of the directory: ", os.listdir(fm_path))
training_dataset = tf.data.Dataset.load(path=os.path.join(fm_path, "training_dataset"), reader_func=custom_reader_func)
validation_dataset = tf.data.Dataset.load(path=os.path.join(fm_path, "validation_dataset"), reader_func=custom_reader_func)
testing_dataset = tf.data.Dataset.load(path=os.path.join(fm_path, "test_dataset"), reader_func=custom_reader_func)

current directory:  /mnt/c/Users/mikef/git/astro_iqa/src/modeling
Content of the directory:  ['.gitattributes', 'map_images_labels.json', 'map_images_labels_cadc.json', 'map_images_labels_cadc2.json', 'map_images_labels_ngc0869.json', 'map_images_labels_ngc0896.json', 'map_images_labels_ngc7000.json', 'modelling.md', 'objects_catalog_cadc2.parquet.gz', 'objects_catalog_ngc0869.parquet.gz', 'objects_catalog_ngc0896.parquet.gz', 'objects_catalog_ngc7000.parquet.gz', 'test_dataset', 'training_dataset', 'validation_dataset']


2025-03-21 13:45:41.121858: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-21 13:45:41.422872: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-21 13:45:41.422936: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-21 13:45:41.426715: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-21 13:45:41.426791: I external/local_xla/xla/stream_executor

In [5]:
# As we separated the labels from the features, we need to convert the labels to StringLookups
label_lookup = tf.keras.layers.StringLookup()
label_lookup.adapt(training_dataset.map(lambda x, y: y))

training_dataset = training_dataset.map(lambda x, y: (x, label_lookup(y)))
validation_dataset = validation_dataset.map(lambda x, y: (x, label_lookup(y)))
testing_dataset = testing_dataset.map(lambda x, y: (x, label_lookup(y)))

In [6]:
from tensorflow.keras.layers import StringLookup, Normalization
from tensorflow.keras.utils import Sequence


#

Testing our datasets for sanity checks

In [7]:

[(train_features, label_batch)] = training_dataset.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of ELLIPTICITY:', train_features['ELLIPTICITY'])
print('A batch of targets:', label_batch )

Every feature: ['ISO0', 'FITS_ID', 'FLAGS', 'ELLIPTICITY', 'CCD_ID', 'CLASS_STAR', 'gt_label1', 'ELONGATION', 'EXPTIME', 'BACKGROUND']
A batch of ELLIPTICITY: tf.Tensor(
[[0.78264153]
 [0.47585982]
 [0.6671052 ]
 [0.17121279]
 [0.76049113]
 [0.7568648 ]
 [0.34455228]
 [0.7747539 ]
 [0.5960636 ]
 [0.51149046]
 [0.861753  ]
 [0.80339986]
 [0.29815495]
 [0.9918808 ]
 [0.9840157 ]
 [0.50382775]
 [0.9969438 ]
 [0.30246472]
 [0.72142637]
 [0.78332305]
 [0.05240852]
 [0.71980107]
 [0.6715517 ]
 [0.3731296 ]
 [0.7682065 ]
 [0.64978206]
 [0.0649904 ]
 [0.57324195]
 [0.7616688 ]
 [0.8111505 ]
 [0.7778587 ]
 [0.82150084]
 [0.84908044]
 [0.36903435]
 [0.7625987 ]
 [0.87358457]
 [0.50443614]
 [0.71031165]
 [0.8691399 ]
 [0.4297365 ]
 [0.5382836 ]
 [0.61038566]
 [0.57938814]
 [0.3192072 ]
 [0.9550641 ]
 [0.66174954]
 [0.78321755]
 [0.8423984 ]
 [0.63916516]
 [0.7373588 ]
 [0.8843243 ]
 [0.7907473 ]
 [0.6789378 ]
 [0.9104991 ]
 [0.5105925 ]
 [0.7571713 ]
 [0.43734515]
 [0.41002744]
 [0.7346002 ]
 [0.

In [8]:

[(validation_features, label_batch)] = validation_dataset.take(1)
print('Every feature:', list(validation_features.keys()))
print('A batch of ELLIPTICITY:', validation_features['ELLIPTICITY'])
print('A batch of targets:', label_batch )

Every feature: ['ISO0', 'FITS_ID', 'FLAGS', 'ELLIPTICITY', 'CCD_ID', 'CLASS_STAR', 'gt_label1', 'ELONGATION', 'EXPTIME', 'BACKGROUND']
A batch of ELLIPTICITY: tf.Tensor(
[[0.35624993]
 [0.26592666]
 [0.83772945]
 [0.7027589 ]
 [0.88597155]
 [0.7756514 ]
 [0.21023828]
 [0.7262485 ]
 [0.34354144]
 [0.6349837 ]
 [0.72408426]
 [0.59848714]
 [0.42157894]
 [0.866242  ]
 [0.5675301 ]
 [0.5993735 ]
 [0.2283591 ]
 [0.527534  ]
 [0.3625645 ]
 [0.31038666]
 [0.7874818 ]
 [0.5818038 ]
 [0.37114233]
 [0.4620301 ]
 [0.8003256 ]
 [0.64659506]
 [0.6382233 ]
 [0.47726828]
 [0.45862538]
 [0.18544888]
 [0.76987904]
 [0.39453256]
 [0.7794554 ]
 [0.5655196 ]
 [0.46227401]
 [0.645999  ]
 [0.2810517 ]
 [0.6341995 ]
 [0.6006255 ]
 [0.6220186 ]
 [0.870293  ]
 [0.6359205 ]
 [0.33743584]
 [0.31926966]
 [0.3068831 ]
 [0.6903769 ]
 [0.6277549 ]
 [0.88080883]
 [0.67995584]
 [0.5032791 ]
 [0.834937  ]
 [0.7598072 ]
 [0.9273336 ]
 [0.7901139 ]
 [0.57319057]
 [0.47191143]
 [0.07389432]
 [0.57480085]
 [0.7690511 ]
 [0.

In [9]:
print("testing dataset elements: ", testing_dataset.element_spec)
[(test_features, label_batch)] = testing_dataset.take(1)
print('Every feature:', list(test_features.keys()))
print('A batch of ELLIPTICITY:', test_features['ELLIPTICITY'])
# print("a batch of features: ", test_features)
print('A batch of targets:', label_batch )


testing dataset elements:  ({'ISO0': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'FITS_ID': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'FLAGS': TensorSpec(shape=(None, 1), dtype=tf.int16, name=None), 'ELLIPTICITY': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'CCD_ID': TensorSpec(shape=(None, 1), dtype=tf.uint8, name=None), 'CLASS_STAR': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'gt_label1': TensorSpec(shape=(None, 1), dtype=tf.string, name=None), 'ELONGATION': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'EXPTIME': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'BACKGROUND': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))
Every feature: ['ISO0', 'FITS_ID', 'FLAGS', 'ELLIPTICITY', 'CCD_ID', 'CLASS_STAR', 'gt_label1', 'ELONGATION', 'EXPTIME', 'BACKGROUND']
A batch of ELLIPTICITY: tf.Tensor(
[[0.73774564]
 [0.76792455]
 [0.88322127]
 [0.5865227

In [10]:
# Fonction pour séparer les features et les labels
# def prepare_data(features, labels):
#     gt_label1 = labels['gt_label1']  # Récupérer la colonne cible
#     return features, gt_label1
# training_dataset = training_dataset.map(prepare_data)
# validation_dataset = validation_dataset.map(prepare_data)
# testing_dataset = testing_dataset.map(prepare_data)
# Prefetch the data
training_dataset = training_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
testing_dataset = testing_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# from data_acquisition_understanding.dnn_datasets_preparation import read_and_concat_catalogs
# test_df = read_and_concat_catalogs([os.path.join(fm_path, "objects_catalog_ngc0869.parquet.gz")])

In [11]:
FEATURE_NAMES = [
    # "OBJECT_ID", # object
    "FITS_ID", # object
    "CCD_ID", # uint8
    "ISO0", # float32
    "BACKGROUND", # float32
    "ELLIPTICITY", # float32
    "ELONGATION", # float32
    "CLASS_STAR", # float32
    "FLAGS", # int16
    "EXPTIME", # float32
    "gt_label1" # object
]

NUMERIC_FEATURE_NAMES = {
    "ISO0": "float32",
    "BACKGROUND": "float32",
    "ELLIPTICITY": "float32",
    "ELONGATION": "float32",
    "CLASS_STAR": "float32",
    "EXPTIME": "float32"
}

CATEGORICAL_FEATURE_NAMES = {
    "FITS_ID": "string",
    "CCD_ID": "uint8",
    "FLAGS": "int16",
    "gt_label1": "string"
}

ID_COLUMNS = {
    "OBJECT_ID": "string"
}

## Create model inputs

In [12]:
def get_normalization_layer(name, dataset):
    print("Processing numerical feature: ", name)
    # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

In [13]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # Create a layer that turns strings into integer indices.
    print("Processing categorical feature: ", name)
    if dtype == 'string':
        print(" ... StringLookup")
        index = layers.StringLookup(max_tokens=max_tokens)
    # Otherwise, create a layer that turns integer values into integer indices.
    else:
        print(" ... IntegerLookup")
        index = layers.IntegerLookup(max_tokens=max_tokens)

    # Prepare a `tf.data.Dataset` that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)

    # Encode the integer indices.
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    return lambda feature: encoder(index(feature))

## Encode input features

For categorical features, we encode them using `layers.Embedding` using the
`encoding_size` as the embedding dimensions. For the numerical features,
we apply linear transformation using `layers.Dense` to project each feature into
`encoding_size`-dimensional vector. Thus, all the encoded features will have the
same dimensionality.

In [14]:
def encode_inputs(dataset, numeric_headers, categorical_headers):

    all_inputs = {}
    encoded_features = []

    # Numerical features.
    for feat_name, dtype in numeric_headers.items():
        numeric_col = tf.keras.Input(shape=(1,), name=feat_name)
        normalization_layer = get_normalization_layer(feat_name, dataset)
        encoded_numeric_col = normalization_layer(numeric_col)
        all_inputs[feat_name] = numeric_col
        encoded_features.append(encoded_numeric_col)


    for feat_name, dtype in categorical_headers.items():
        categorical_col = tf.keras.Input(shape=(1,), name=feat_name, dtype=dtype)
        encoding_layer = get_category_encoding_layer(name=feat_name,
                                                    dataset=dataset,
                                                    dtype=dtype,
                                                    max_tokens=10)
        encoded_categorical_col = encoding_layer(categorical_col)
        all_inputs[feat_name] = categorical_col
        encoded_features.append(encoded_categorical_col)
    

    return all_inputs, encoded_features

In [15]:
# testing the function
all_inputs, encoded_features = encode_inputs(validation_dataset, NUMERIC_FEATURE_NAMES, CATEGORICAL_FEATURE_NAMES)

print("encoded features dypes: ", [x.dtype for x in encoded_features])

Processing numerical feature:  ISO0


2025-03-21 13:48:45.414709: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Processing numerical feature:  BACKGROUND
Processing numerical feature:  ELLIPTICITY
Processing numerical feature:  ELONGATION
Processing numerical feature:  CLASS_STAR
Processing numerical feature:  EXPTIME
Processing categorical feature:  FITS_ID
 ... StringLookup
Processing categorical feature:  CCD_ID
 ... IntegerLookup
Processing categorical feature:  FLAGS
 ... IntegerLookup
Processing categorical feature:  gt_label1
 ... StringLookup
encoded features dypes:  [tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32]


In [29]:
[(test_features, label_batch)] = testing_dataset.take(1)
test_ellip = test_features['ELLIPTICITY']
test_encod_ellip = get_normalization_layer("ELLIPTICITY", testing_dataset)
test_encod_ellip(test_ellip)

Processing numerical feature:  ELLIPTICITY


<tf.Tensor: shape=(128, 1), dtype=float32, numpy=
array([[ 1.2284503 ],
       [ 1.3497897 ],
       [ 1.8133597 ],
       [ 0.6204332 ],
       [-1.0678602 ],
       [ 0.32637793],
       [ 0.97721297],
       [ 1.406932  ],
       [ 1.9609343 ],
       [ 1.590036  ],
       [ 1.1273293 ],
       [ 1.6815034 ],
       [ 1.2027459 ],
       [-1.0518092 ],
       [-0.2405533 ],
       [ 0.02042446],
       [ 1.1894319 ],
       [ 1.4150672 ],
       [ 0.4331511 ],
       [ 0.93883413],
       [-0.17082123],
       [ 1.3351921 ],
       [ 0.13287951],
       [-0.79700536],
       [ 1.172977  ],
       [-0.37903488],
       [ 0.29318678],
       [ 1.9395019 ],
       [ 0.13663316],
       [ 1.3452564 ],
       [ 1.0009425 ],
       [ 0.7916211 ],
       [ 0.20184423],
       [-0.0476176 ],
       [ 0.95010537],
       [ 0.7475537 ],
       [ 0.11652456],
       [ 0.24605568],
       [ 0.7201779 ],
       [-1.1036808 ],
       [ 1.51507   ],
       [ 0.5456349 ],
       [ 0.8753409 ],
    

In [30]:
test_gt_label = test_features['gt_label1']
test_encod_gt_label = get_category_encoding_layer("gt_label1", testing_dataset, "string", 10)
test_encod_gt_label(test_gt_label)

Processing categorical feature:  gt_label1
 ... StringLookup


<tf.Tensor: shape=(128, 6), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.

## Implement the Gated Linear Unit

[Gated Linear Units (GLUs)](https://arxiv.org/abs/1612.08083) provide the
flexibility to suppress input that are not relevant for a given task.

In [16]:

class GatedLinearUnit(layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.linear = layers.Dense(units)
        self.sigmoid = layers.Dense(units, activation="sigmoid")

    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)


## Implement the Gated Residual Network

The Gated Residual Network (GRN) works as follows:

1. Applies the nonlinear ELU transformation to the inputs.
2. Applies linear transformation followed by dropout.
4. Applies GLU and adds the original inputs to the output of the GLU to perform skip
(residual) connection.
6. Applies layer normalization and produces the output.

In [17]:

class GatedResidualNetwork(layers.Layer):
    def __init__(self, units, dropout_rate):
        super().__init__()
        self.units = units
        self.elu_dense = layers.Dense(units, activation="elu")
        self.linear_dense = layers.Dense(units)
        self.dropout = layers.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = layers.LayerNormalization()
        self.project = layers.Dense(units)

    def call(self, inputs):
        x = self.elu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x


## Implement the Variable Selection Network

The Variable Selection Network (VSN) works as follows:

1. Applies a GRN to each feature individually.
2. Applies a GRN on the concatenation of all the features, followed by a softmax to
produce feature weights.
3. Produces a weighted sum of the output of the individual GRN.

Note that the output of the VSN is [batch_size, encoding_size], regardless of the
number of the input features.

In [18]:

class VariableSelection(layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        # MFT: in our case, we will use (or try at least) to use the categorical features as this without
        # embeddings, as they don't have a high dimensionality. One represent the file name, another the CCD_ID between
        # 1 and 16, the target represents 5 different classes, and the flags can take few values as well.
        v = layers.concatenate(inputs)
        v = self.grn_concat(v)
        # v = keras.ops.expand_dims(self.softmax(v), axis=-1)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input in enumerate(inputs):
            x.append(self.grns[idx](input))
        # x = keras.ops.stack(x, axis=1)
        x = tf.stack(x, axis=1)

        # outputs = keras.ops.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs


## Create Gated Residual and Variable Selection Networks model

In [19]:
def create_model(dataset, numeric_headers, categorical_headers,  
                 encoding_size=32, dropout_rate=0.15):
    """
    Create a model with embedding layers for categorical features.
    """
    all_inputs, encoded_inputs = encode_inputs(dataset, numeric_headers, categorical_headers)

    num_features = len(all_inputs)
    print("Total number of features:", num_features)
    vsn = VariableSelection(num_features, encoding_size, dropout_rate)(
        encoded_inputs
    )

    outputs = layers.Dense(units=1, activation="sigmoid")(vsn)
    model = keras.Model(inputs=all_inputs, outputs=outputs)

    return model


## Compile, train, and evaluate the model

In [20]:
learning_rate = 0.001
dropout_rate = 0.15
batch_size = 265
num_epochs = 20
encoding_size = 16

model = create_model(training_dataset, NUMERIC_FEATURE_NAMES, CATEGORICAL_FEATURE_NAMES, 
                 encoding_size, dropout_rate)





Processing numerical feature:  ISO0
Processing numerical feature:  BACKGROUND
Processing numerical feature:  ELLIPTICITY
Processing numerical feature:  ELONGATION
Processing numerical feature:  CLASS_STAR
Processing numerical feature:  EXPTIME
Processing categorical feature:  FITS_ID
 ... StringLookup
Processing categorical feature:  CCD_ID
 ... IntegerLookup
Processing categorical feature:  FLAGS
 ... IntegerLookup
Processing categorical feature:  gt_label1
 ... StringLookup
Total number of features: 10


In [21]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [22]:
# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

print("Start training the model...")

model.fit(
    training_dataset,
    epochs=num_epochs,
    validation_data=validation_dataset,
    callbacks=[early_stopping],
)
print("Model training finished.")



Start training the model...
Epoch 1/20


2025-03-21 13:58:24.936624: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2025-03-21 13:58:25.264321: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fbd984a9b00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-03-21 13:58:25.264375: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro P4000, Compute Capability 6.1
2025-03-21 13:58:25.295169: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742561905.467739   34577 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11926/11926 [==============================] - 813s 67ms/step - loss: nan - accuracy: 3.5377e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/20
  133/11926 [..............................] - ETA: 12:34 - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
print("Evaluating model performance...")
_, accuracy = model.evaluate(testing_dataset)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

You should achieve more than 95% accuracy on the test set.

To increase the learning capacity of the model, you can try increasing the
`encoding_size` value, or stacking multiple GRN layers on top of the VSN layer.
This may require to also increase the `dropout_rate` value to avoid overfitting.

**Example available on HuggingFace**

| Trained Model | Demo |
| :--: | :--: |
| [![Generic badge](https://img.shields.io/badge/%F0%9F%A4%97%20Model-Classification%20With%20GRN%20%26%20VSN-red)](https://huggingface.co/keras-io/structured-data-classification-grn-vsn) | [![Generic badge](https://img.shields.io/badge/%F0%9F%A4%97%20Space-Classification%20With%20GRN%20%26%20VSN-red)](https://huggingface.co/spaces/keras-io/structured-data-classification-grn-vsn) |